In [9]:
import random
import pandas as pd
import spacy
from spacy import displacy
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.scorer import Scorer

import nlp_project_functions as functions

In [2]:
df = pd.read_csv("data/train_test_val/test.tsv", sep="\t", skip_blank_lines=False, names=["TOKEN", "ENT"])

In [3]:
tokens = df["TOKEN"].to_list()
sentences = functions.make_sentences(tokens)

# create a test sentence for a quick check of the model later on
test_sentence = ''.join(sentences[:5])
print(test_sentence)

 Einweihungs Predigt Der Neuen Orgel, Zu Wrietzen an der Oder, Den 1 Advents Sonntag, im Jahr Christi 1729 Aus den Worten des Evangelii Matth 21 v 9 Hosianna dem Sohn David, gelobet sey der da kömmt im Nahmen des Herrn! Hosianna in der Höhe. Jn ansähnlicher und sehr volckreicher Versammlung, Einfältig, doch Schriftmäßig gehalten, Und auf Begehren Der lieben Obrigkeit und Christlichen Gemeine ausgefertiget, Von Carl Etzard Bödikern, Mittelsten Predigern daselbst. Berlin, gedruckt bey Johann Lorentz, Königlich Preußischer privilegierter Buchdrucker. 1730 Deus Optimus Maximus Sanctus. Si Deus omnipotens ea nunc dat gaudia terris; In coelis recreans gaudia quanta dabit! Gönnt Gott den Menschen schon hier solche Freud auf Erden; Wie frölich wird man denn dort in dem Himmel werden? In majorem Dei honorem, laudem und gloriam, ad hujus templi ornamentum, und ad universae concionis excitandam deovtionem, sub felici regiminie Serenissimi atque Potentissimi Domini, Domini Friderici Wilhelmi, Regi

In [4]:
nlp_default = spacy.load("de_core_news_sm")
doc_default = nlp_default(test_sentence)
spacy_default_ents = []
for ent in doc_default.ents:
    spacy_default_ents.append((ent.text, ent.label_))

print(spacy_default_ents)

[('Der Neuen Orgel', 'MISC'), ('Wrietzen an der Oder', 'LOC'), ('Den 1 Advents Sonntag', 'MISC'), ('im Jahr Christi', 'MISC'), ('Evangelii Matth', 'PER'), ('Sohn David', 'PER'), ('gelobet sey', 'PER'), ('Herrn!', 'ORG'), ('Der lieben Obrigkeit', 'MISC'), ('Christlichen Gemeine ausgefertiget', 'MISC'), ('Carl Etzard Bödikern', 'PER'), ('Berlin', 'LOC'), ('Johann Lorentz', 'PER'), ('Königlich Preußischer', 'PER'), ('Deus Optimus Maximus Sanctus', 'PER'), ('Si Deus', 'PER'), ('coelis recreans', 'LOC'), ('regiminie Serenissimi', 'PER'), ('Potentissimi Domini', 'PER'), ('Domini Friderici Wilhelmi', 'PER'), ('Regis Borussiae', 'PER'), ('Electoris Brandenburgici', 'PER'), ('c', 'MISC'), ('c', 'MISC'), ('constans stanneis', 'PER'), ('aeneis', 'MISC'), ('fistulis ac', 'LOC'), ('Joachimo VVagnero', 'PER'), ('Artifice', 'LOC'), ('hac re excellentissimo', 'LOC')]


In [6]:
def create_training_data(path: str) -> list:
    # see https://github.com/explosion/spaCy/discussions/10717#discussioncomment-2652145
    doc_bin = DocBin().from_disk(path)
    nlp = spacy.blank("de")
    
    examples = []
    for doc in doc_bin.get_docs(nlp.vocab):
        entities = []
        for ent in doc.ents:
            entities.append((ent.start_char, ent.end_char, ent.label_))
        
        spacy_entry = (doc.text, {"entities": entities})
        examples.append(spacy_entry)
    return examples

In [7]:
training_data = create_training_data("./data/train_test_val/train.spacy")
validation_data = create_training_data("./data/train_test_val/val.spacy")

In [8]:
training_data[0]

('Christliche Predigt , Welche Bey der sogenannten Einweyhung der neuen Orgel In der Evangelischen Haupt Kirch zu Straßburg Sonntags den 16 Novembris 1749 gehalten worden Von Johann Leonhard Fröreißen , Der Heiligen Schrifft D und Pastore Primario des Collegiat Stiffts zu St Thomä Canonico , Eines Ehrwürdigen Kirchen Convents Praeside und Pastore Primario . Straßburg , Gedruckt bey Melchior Pauschinger . Dem Frey Hoch Wohlgebohrnen Herrn Herrn Frantz Carl Bock von Bläsheim , Der Stadt Straßburg Hochgebietenden Städtmeister und Dreyzehner , der Universität Cancellario Eminentissimo und des Löblichen Stiffts zum Frauen Hauß Pflegern , Denen Hoch Edelgebohrnen , Gestreng , Vest , Fromm , Fürsichtig , Hochweiß und Hochgelehrten Herren Herrn Johann Friderich Hammerer , Der Stadt Straßburg Hochverdienten alten Ammeister und Dreyzehner , der Universität Scholarchae , und der Evangelischen Haupt Kirch Ober Pfleger . Herrn Johann Heinrich Faber , Der Stadt Straßburg Hochverdienten alten Ammeist

In [12]:
# https://stackoverflow.com/a/68346007
def evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annot in examples:
        pred = ner_model(input_)
        #print(pred,annot)
        temp = Example.from_dict(pred, dict.fromkeys(annot))
        example.append(temp)
    scores = scorer.score(example)
    return scores


# see https://www.youtube.com/watch?v=7Z1imsp6g10&list=PL2VXyKi-KpYs1bSnT8bfMFyGS-wMcjesM&index=9
# and https://github.com/explosion/spaCy/discussions/6316#discussioncomment-186347
def train_spacy(training_data, validation_data, iterations, validate = 5):
    TRAIN_DATA = training_data
    VALIDATION_DATA = validation_data

    nlp = spacy.blank("de")

    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)

    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
        
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()

        for itn in range(iterations):
            print(f"Starting Iteration {itn}")
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                examples = []
                examples.append(Example.from_dict(nlp.make_doc(text), annotations))
                nlp.update(examples,
                           drop=0.2,
                           sgd=optimizer,
                           losses=losses)
            print(f"Losses: {losses}")

            if (itn + 1) % validate == 0:
                print("Validation:")
                validation_loss = evaluate(nlp, VALIDATION_DATA)
                print("Validation Loss:", validation_loss)

    return (nlp)

In [13]:
nlp = train_spacy(training_data, validation_data, 60)

Starting Iteration 0
Losses: {'ner': 9658.446372553444}
Starting Iteration 1
Losses: {'ner': 5760.314741235291}
Starting Iteration 2
Losses: {'ner': 5023.408548105032}
Starting Iteration 3
Losses: {'ner': 4651.620348172685}
Starting Iteration 4
Losses: {'ner': 4421.249078434685}
Validation:
Validation Loss: {'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'sents_p': None, 'sents_r': None, 'sents_f': None, 'tag_acc': None, 'pos_acc': None, 'morph_acc': None, 'morph_micro_p': None, 'morph_micro_r': None, 'morph_micro_f': None, 'morph_per_feat': None, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'ents_p': None, 'ents_r': None, 'ents_f': None, 'ents_per_type': None, 'cats_score': 0.0, 'cats_score_desc': 'macro F', 'cats_micro_p': 0.0, 'cats_micro_r': 0.0, 'cats_micro_f': 0.0, 'cats_macro_p': 0.0, 'cats_macro_r': 0.0, 'cats_macro_f': 0.0, 'cats_macro_auc': 0.0, 'cats_f_per_type': {}, 'cats_auc_per_type': {}}
Starting Iteration 5
Losses: {'ner': 4168.96105718

In [15]:
nlp.to_disk("./models/sermons_60it")

In [16]:
nlp_custom = spacy.load("models/sermons_60it")

In [17]:
doc = nlp_custom(test_sentence)
spacy_custom_ents = []
for ent in doc.ents:
    spacy_custom_ents.append((ent.text, ent.label_))

print(spacy_custom_ents)

[('Einfältig', 'PER'), ('Etzard', 'PER'), ('Johann Lorentz', 'PER'), ('Friderici Wilhelmi', 'PER'), ('Artifice', 'PER')]


In [19]:
displacy.render(doc, style="ent", jupyter=True)

In [20]:
displacy.render(doc_default, style="ent")